In [121]:
import csv
import re
import spacy
from spacy import displacy

import urllib
from owlready2 import *
import rdflib

import re

from allennlp.common.testing import AllenNlpTestCase
from allennlp.predictors.predictor import Predictor

# pre-processing
def PreProcess(senSet):
    #remove content between [ ]
    print("Pre-processing...")
    for index in range(len(senSet)):
        while senSet[index].find('[')>=0:
            i_start = senSet[index].find('[')
            i_end = senSet[index].find(']')
            s = senSet[index][i_start:i_end+2]
            senSet[index] = senSet[index].replace(s, "")
            
# stopwords from parsing the whole sentence
def RemoveStopword1(phrase, doc, chunkStart, chunkEnd, stopList):
    result = phrase
    i_stop=0
    #start = chunk.start# to eliminate the condition when the first word of chunk is stop word
    for i_sen in range(chunkStart, chunkEnd):
        while i_stop < len(stopList) and stopList[i_stop] < i_sen-1:
            #print(str(stopList[i_stop]) + ' ' + str(i_sen))
            i_stop = i_stop+1
        # there is no stop word in current chunk
        if i_stop >= len(stopList):
            break;
        #print(i_sen)
        # finish going through the chunk
        if stopList[i_stop] > chunkEnd-1:
            break
        # find the stop word and remove it
        if stopList[i_stop] == i_sen-1:
            #print(doc[i_sen-1])
            if i_sen-1 == chunkStart:
                result = result.replace(doc[i_sen-1].text + ' ', '')
                chunkStart = chunkStart+1
            else:
                result = result.replace(' ' + doc[i_sen-1].text, '')
    return result

# stopwords from parsing triple separately
def RemoveStopword2(inputPhrase):
    result = ''
    doc_phrase = nlp(str(inputPhrase))
    for token in doc_phrase:
        #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
        #       token.shape_, token.is_alpha, token.is_stop)
        if not token.is_stop:
            result = result + token.text + ' '
        #else:
        #    print(token.text + ', ', end = '')    
    return result


# extract one triple from given sentence
def ExtractTriple(sen):
    # initialize the triple and stop word list
    subj = ""
    pred = ""
    obj = ""
    stopList = []
    
    # parse sentence
    doc = nlp(str(sen))
    print('\n' + str(index) + ': ' + senSet[index])
    
    ## visualize the semantic tree
    #options = {'compact': True, 'color': 'blue'}
    #displacy.serve(doc, style='dep', options=options)
    #displacy.serve(doc, style='dep')

    print('stop words: ', end='')
    for token in doc:
        #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
        #      token.shape_, token.is_alpha, token.is_stop)

        # record the index of stop words
        if token.is_stop:
            print(token.text + ', ', end='')
            stopList.append(token.i)
        if re.match('nsubj', token.dep_):   
            subj = token.text
        if re.match('ROOT', token.dep_): 
            pred = token.lemma_
            pred_orig = token.text
        if re.match('dobj', token.dep_): 
            obj = token.text
            '''#an earlier solution that I find not necessary
            obj = token.lemma_
            # to avoid cases like "-PRON-"
            if obj[0] == '-':
                obj = token.text'''
    print('\n')

    subj_1 = subj
    obj_1 = obj
    # using chunk to update subject and object
    for chunk in doc.noun_chunks:
        if chunk.root.head.text == pred_orig and re.match('nsubj', chunk.root.dep_):
            subj = chunk.text
            # remove stop words
            subj_1 = RemoveStopword1(subj, doc, chunk.start, chunk.end, stopList)

        if chunk.root.head.text == pred_orig and re.match('dobj|attr', chunk.root.dep_):
            obj = chunk.text
            # remove stop words
            obj_1 = RemoveStopword1(obj, doc, chunk.start, chunk.end, stopList)
        #print(chunk.text + ' ' + str(chunk.start))
        #print(chunk.text, chunk.root.text, chunk.root.dep_, chunk.root.head.text)

    #print('Before : ' + subj + ' - ' + pred + ' - ' + obj)
    #print('Method1: ' + subj_1 + ' - ' + pred + ' - ' + obj_1)

    # second method to remove stop words
    subj_2 = RemoveStopword2(subj)
    obj_2 = RemoveStopword2(obj)
    #print('Method2: ' + subj_2 + '- ' + pred + ' - ' + obj_2 + '\n')

    return [subj, pred, obj]

# transfer a phrase to a URI form
def FormatURI(phrase, isS_O = False):
    #print('Before formatting:  ' + phrase)
    chars = list(phrase)
    
    if len(chars) > 0 and not isS_O:
        chars[0] = chars[0].upper()
    for i in range(len(chars)):
        if chars[i] == ' ' and i+1 < len(chars):
            chars[i+1] = chars[i+1].upper()
    phrase = ''.join(chars)
    phrase = phrase.replace(' ', '')
    phrase = re.sub(r'[^a-zA-Z0-9\s]', '', phrase)
    #print('After formatting:  ' + phrase)
    return phrase

# query the given triple in the ontology with SPARQL
# return true/false as result
def QueryTriple(subj, pred, obj):
    prefix = """
    PREFIX rdf:<http://www.w3.org/2000/01/rdf-schema#>
    PREFIX dbpd:<http://dbpedia.org/ontology/>
    """
    #subj = "provinceLink"
    #pred = "range"
    #obj = "Province"
    qSelect = prefix + """
    SELECT ?sub WHERE {
      ?sub rdf:""" + FormatURI(pred) + """ dbpd:""" + FormatURI(obj) + """.
    }"""
    qAsk = prefix + """
    ASK {
        dbpd:""" + FormatURI(subj) + """ rdf:""" + FormatURI(pred) + """ dbpd:""" + FormatURI(obj) + """.
    }"""
    
    r = list(m_graph.query(qAsk))
    return r

def ComponentQuery1(subj, pred, obj):
    prefix = """
    PREFIX rdf:<http://www.w3.org/2000/01/rdf-schema#>
    PREFIX dbpd:<http://dbpedia.org/ontology/>
    """
    #subj = "provinceLink"
    #pred = "range"
    #obj = "province"
    
    qSelect_P_O = prefix + """
    SELECT ?pred ?obj WHERE {
      dbpd:""" + FormatURI(subj, True) + """ ?pred ?obj.
    }"""
    
    qSelect_S_O = prefix + """
    SELECT ?sub ?obj WHERE {
      ?sub rdf:""" + FormatURI(pred, True) + """ ?obj.
    }"""
    
    qSelect_S_P = prefix + """
    SELECT ?sub ?pred WHERE {
      ?sub ?pred dbpd:""" + FormatURI(obj) + """.
    }"""
    
    #print(qSelect_S_P)
    # may need one more variable to record source
    r = list(m_graph.query(qSelect_P_O))
    r.append(list(m_graph.query(qSelect_S_O)))
    r.append(list(m_graph.query(qSelect_S_P)))
    
    #if r!=[]:
    #    print(r)
    return r

def ComponentQuery2(subj, pred, obj):
    prefix = """
    PREFIX rdf:<http://www.w3.org/2000/01/rdf-schema#>
    PREFIX dbpd:<http://dbpedia.org/ontology/>
    """
    #subj = "provinceLink"
    #pred = "range"
    #obj = "province"
    
    qSelect_S = prefix + """
    SELECT ?sub WHERE {
      ?sub rdf:""" + FormatURI(pred, True) + """ dbpd:""" + FormatURI(obj) + """.
    }"""
    
    qSelect_P = prefix + """
    SELECT ?pred WHERE {
      dbpd:""" + FormatURI(subj, True) + """ ?pred dbpd:""" + FormatURI(obj) + """.
    }"""
    
    qSelect_O = prefix + """
    SELECT ?obj WHERE {
      dbpd:""" + FormatURI(subj, True) + """ rdf:""" + FormatURI(pred, True) + """ ?obj.
    }"""
    
    #print(qSelect_S)
    # may need one more variable to record source
    r = list(m_graph.query(qSelect_S))
    r.append(list(m_graph.query(qSelect_P)))
    r.append(list(m_graph.query(qSelect_O)))
    
    #if r!=[]:
    #    print(r)
    return r

def ComponentQuery3(subj, pred, obj):
    prefix = """
    PREFIX rdf:<http://www.w3.org/2000/01/rdf-schema#>
    PREFIX dbpd:<http://dbpedia.org/ontology/>
    """
    #subj = "provinceLink"
    #pred = "range"
    #obj = "province"
    
    qAsk = prefix + """
    ASK {
        dbpd:""" + FormatURI(subj, True) + """ rdf:""" + FormatURI(pred, True) + """ dbpd:""" + FormatURI(obj) + """.
    }"""
    
    r = list(m_graph.query(qAsk))    
    
    if not r:
        print(qAsk)
        print(r)
    return r

def PartialQuery(subj, pred, obj):
    doc_subj = nlp(str(subj))
    doc_pred = nlp(str(pred))
    doc_obj = nlp(str(obj))
    r1 = []
    r2 = []
    r3 = False
    for token_subj in doc_subj:
        for token_pred in doc_pred:
            for token_obj in doc_obj:
                #print(token_obj.text, token_obj.lemma_, token_obj.pos_, token_obj.tag_, token_obj.dep_,
                #      token_obj.shape_, token_obj.is_alpha, token_obj.is_stop)
                if not token_subj.is_stop or not token_pred.is_stop or not token_obj.is_stop:
                    r1 = ComponentQuery1(token_subj.text, token_pred.text, token_obj.text)
                    if r1 and r1[0]:
                        r2 = ComponentQuery2(token_subj.text, token_pred.text, token_obj.text)
                        if r2 and r2[0]:
                            r3 = ComponentQuery3(token_subj.text, token_pred.text, token_obj.text)
                            if not r3:
                                print("Find 3 component:")
                                print(str(token_subj.text) + " - " + str(token_pred.text) + " - " + str(token_obj.text))
                                break
                            else:
                                print("Only find 2 component:")
                                print(r2)
                        else:
                            print("Only find 1 component:")
                            print(r1)
                else:
                    print("All query words are stop words!")
    return r3

# load Spacy NLP dictionary
nlp = spacy.load('en_core_web_sm')

# load DBPD ontology and construct graph for query
m_world = World()# Owlready2 stores every triples in a ‘World’ object
m_onto = m_world.get_ontology("dbpedia.owl").load()
m_graph = m_world.as_rdflib_graph()

# load data
file = open("shortdataset.csv", "r")
#file = open("newdataset_formatted.csv", "r")
reader = csv.reader(file)
senSet = []
for item in reader:
    #format sentences in item as string
    fullP = "".join(item)
    splitP = fullP.split(";", 3);
    splitS = splitP[3][1:len(splitP[3])].split(".");
    #print(splitS)
    for sen in splitS:
        senSet.append(sen)#store the sentence into an array
file.close()
print("Total sentences: " + str(len(senSet)))

# pre-processing
PreProcess(senSet)

# parse and query each sentence
#for index in range(len(senSet)):
index = 2

#for word, tag in zip(results["words"], results["tags"]):
#    print(f"{word}\t{tag}")

# extract triple from current sentence
[subj, pred, obj] = ExtractTriple(senSet[index])
print('Triple for Query   : ' + subj + ' - ' + pred + ' - ' + obj)

# parse with AllenNLP
'''from allennlp.predictors import Predictor
predictor = Predictor.from_path("srl-model-2018.05.25.tar.gz")
results = predictor.predict(senSet[index])
for verb in zip(results["verbs"]):
    print(f"{verb}")
#for word, verb in zip(results["words"], results["verbs"]):
#    print(f"{word}\t{verb}")
'''
    
# query the triple in dbpd with SPARQL
queryResult = QueryTriple(subj, pred, obj)
# print('Triple Query Result: ' + str(queryResult))

#subj = "province link"
#pred = "range"
#obj = "province"


# look up the URI for subj, pred and obj
localSite = 'http://localhost:1111/api/search/KeywordSearch?'
onlineSite = 'http://lookup.dbpedia.org/api/search/KeywordSearch?'
response = urllib.request.urlopen(onlineSite + 
'QueryClass=' + '' + 
'&MaxHits=' + '5' +
'&QueryString=' + subj).read()
print(response)
print(type(response))

# query the triple with several different methods
'''r = ComponentQuery3(subj, pred, obj)
if r and r[0]:
    print("Find origin component:")
    print(subj + ' - ' + pred + ' - ' + obj)
else:
    PartialQuery(subj, pred, obj)'''

* Owlready2 * WARNING: http://dbpedia.org/ontology/senator belongs to more than one entity types (e.g. Class, Property, Individual): [owl.ObjectProperty, dbpedia.MemberOfParliament, DUL.sameSettingAs]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/politicGovernmentDepartment belongs to more than one entity types (e.g. Class, Property, Individual): [owl.ObjectProperty, dbpedia.Department, DUL.hasPart]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/productShape belongs to more than one entity types (e.g. Class, Property, Individual): [owl.DataProperty, DUL.hasQuality]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/latinName belongs to more than one entity types (e.g. Class, Property, Individual): [owl.DataProperty, dbpedia.Name]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/iso6391Code belongs to more than one entity types (e.g. Class, Property, Individual): [owl.DataPrope

Total sentences: 46
Pre-processing...

2:   Um we manage all the construction and innovation projects for the university across the state
stop words: we, all, the, and, for, the, across, the, 

Triple for Query   : we - manage - all the construction and innovation projects
b'<?xml version="1.0" encoding="utf-8"?>\n<ArrayOfResult xmlns="http://lookup.dbpedia.org/" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">\n      <Result>\n      <Label>United States</Label>\n      <URI>http://dbpedia.org/resource/United_States</URI>\n      <Description>The United States of America (USA), commonly referred to as the United States (U.S.) or America, is a federal republic composed of 50 states, a federal district, five major territories and various possessions. The 48 contiguous states and Washington, D.C., are in central North America between Canada and Mexico. The state of Alaska is located in the northwestern part of North America and the state of

'r = ComponentQuery3(subj, pred, obj)\nif r and r[0]:\n    print("Find origin component:")\n    print(subj + \' - \' + pred + \' - \' + obj)\nelse:\n    PartialQuery(subj, pred, obj)'

In [67]:
# to remove WARNINGs from Owlready2
from IPython.display import HTML
HTML('''<script>
code_show_err=false; 
function code_toggle_err() {
 if (code_show_err){
 $('div.output_stderr').hide();
 } else {
 $('div.output_stderr').show();
 }
 code_show_err = !code_show_err
} 
$( document ).ready(code_toggle_err);
</script>
To toggle on/off output_stderr, click <a href="javascript:code_toggle_err()">here</a>.''')


In [59]:
# reference: https://pythonhosted.org/Owlready2/world.html
from owlready2 import *
import rdflib

my_world = World()# Owlready2 stores every triples in a ‘World’ object
onto = my_world.get_ontology("dbpedia.owl").load()

graph = my_world.as_rdflib_graph()
print(len(graph))

prefix = """
PREFIX rdf:<http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbpd:<http://dbpedia.org/ontology/>
"""

'''r = list(graph.query(prefix + """
SELECT ?sub WHERE {
  ?sub rdf:range  dbpd:Province.
}"""))'''

'''r = list(graph.query(prefix + """
ASK {
  dbpd:provinceLink rdf:range  dbpd:Province.
}"""))'''

r = list(graph.query(prefix + """
DESCRIBE ?sub WHERE {
  ?sub rdf:range  dbpd:Province.
}"""))

print(r)

* Owlready2 * WARNING: http://dbpedia.org/ontology/senator belongs to more than one entity types (e.g. Class, Property, Individual): [owl.ObjectProperty, dbpedia.MemberOfParliament, DUL.sameSettingAs]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/politicGovernmentDepartment belongs to more than one entity types (e.g. Class, Property, Individual): [owl.ObjectProperty, dbpedia.Department, DUL.hasPart]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/productShape belongs to more than one entity types (e.g. Class, Property, Individual): [owl.DataProperty, DUL.hasQuality]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/latinName belongs to more than one entity types (e.g. Class, Property, Individual): [owl.DataProperty, dbpedia.Name]; I'm trying to fix it...


31050


* Owlready2 * WARNING: http://dbpedia.org/ontology/iso6391Code belongs to more than one entity types (e.g. Class, Property, Individual): [owl.DataProperty, dbpedia.LanguageCode]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/iso6393Code belongs to more than one entity types (e.g. Class, Property, Individual): [owl.DataProperty, dbpedia.LanguageCode]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/iso6392Code belongs to more than one entity types (e.g. Class, Property, Individual): [owl.DataProperty, dbpedia.LanguageCode]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/ingredientName belongs to more than one entity types (e.g. Class, Property, Individual): [owl.DataProperty, DUL.hasPart]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/greekName belongs to more than one entity types (e.g. Class, Property, Individual): [owl.DataProperty, dbpedia.Name]; I'm trying to fix it...


Exception: DESCRIBE not implemented

In [ ]:
import Queue